In [1]:
import boto3
import pandas as pd
from io import StringIO , BytesIO
from datetime import datetime , timedelta

## Adapter Layer

In [2]:
def read_csv_to_df(bucket,key,decoding='utf-8',sep=','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_to_s3(bucket,df,key):
    outbuffer =BytesIO()
    df.to_parquet(outbuffer , index=False)
    bucket.put_object(Body = outbuffer.getvalue() , Key = key)
    return True

def return_objects(bucket,arg_date,src_format):
    min_date = datetime.strptime(arg_date ,src_format).date()-timedelta(days=1)
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0],src_format).date() >= min_date]
    return objects
    
    

## Application Layer

In [6]:
# Extract

def extract(bucket,objects):
    df = pd.concat([read_csv_to_df(bucket , obj.key) for obj in objects] , ignore_index=True)
    return df

# Transform

def transform_report1(df,columns,arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform("first")
    df['closing_price'] = df.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform("last")
    df = df.groupby(['ISIN','Date'], as_index=False).agg(opening_price_euro=('opening_price','min'),closing_price_euro=('closing_price','min'), minimun_price_euro=('MinPrice','min'),maximun_price_euro=('MaxPrice','max'), daily_traded_volume=('TradedVolume','sum'))
    df['previous_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_euro'].shift(1)
    df['percentage_change_prev%'] = ((df['closing_price_euro']-df['previous_closing_price'])/df['previous_closing_price'])*100
    df.drop(columns=['previous_closing_price'],inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

# Load

def load(bucket,df,trg_key,trg_format):
    key = trg_key + datetime.today().strftime('%Y-%m-%m_%H-%M-%S') + trg_format
    write_to_s3(bucket,df,key)
    return True

def etl_report1(src_bucket,trg_bucket,objects,columns,arg_date,trg_key,trg_format):
    df = extract(src_bucket,objects)
    df = transform_report1(df,columns,arg_date)
    load(trg_bucket,df,trg_key,trg_format)
    return True
    
    

In [20]:
# main fuction enterypoint

def main():
    # parameters/confugirations
    #read later config
    arg_date = '2022-11-23'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    columns = ['ISIN',  'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume', 'NumberOfTrades']
    trg_bucket = 'targetbucket-1234'
    trg_key = 'xetra_daiy_report_'
    trg_format = '.parquet'
    
    #init connections
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run apllication
    objects = return_objects(bucket_src,arg_date,src_format)
    etl_report1(bucket_src,bucket_trg,objects,columns,arg_date,trg_key,trg_format)
    
    return True
    
    
    
    

In [21]:
# run 

main()

True

## For reading the results

In [22]:
trg_bucket = 'targetbucket-1234'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)

In [23]:
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daiy_report2023-04-04_02-07-12.parquet
xetra_daiy_report_2023-04-04_02-13-42.parquet
xetra_daiy_report_2023-04-04_02-47-29.parquet
xetra_daiy_report_2023-04-04_04-39-19.parquet
xetra_daiy_report_2023-04-04_04-50-38.parquet


In [24]:
prq_obj = bucket_trg.Object(key='xetra_daiy_report_2023-04-04_04-50-38.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [25]:
df_report

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume,percentage_change_prev%
0,AT000000STR1,2022-11-23,37.00,37.95,37.00,37.95,396,2.99
1,AT000000STR1,2022-11-24,36.05,36.35,35.65,37.05,1838,-4.22
2,AT000000STR1,2022-11-25,36.10,37.70,36.10,37.70,2864,3.71
3,AT000000STR1,2022-11-26,36.10,37.70,36.10,37.70,2864,0.00
4,AT000000STR1,2022-11-27,36.10,37.70,36.10,37.70,2864,0.00
...,...,...,...,...,...,...,...,...
125188,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
125189,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
125190,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
125191,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
